In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

## 워드투벡터 데이터 & 전체 데이터 불러오기

In [2]:
data = pd.read_csv('/content/drive/MyDrive/신입프로젝트/final_keyword_scoring_data.csv')

In [3]:
data = data.rename(columns= {'Unnamed: 0' : 'token'})

In [4]:
data

,token,커피/Noun,가격/Noun,직원/Noun,분위기/Noun
0,하다/Verb,1.017735e-13,1.354741e-18,1.293492e-14,1.507433e-15
1,좋다/Adjective,1.440535e-12,5.690393e-18,1.584780e-15,3.057179e-11
2,맛있다/Adjective,8.531844e-13,3.721021e-17,1.747931e-16,1.345982e-13
3,맛/Noun,3.406623e-11,3.650404e-15,1.590326e-16,1.284656e-13
4,커피/Noun,1.000000e+00,9.616869e-16,8.931647e-17,6.098449e-12
...,...,...,...,...,...
2144,비누/Noun,4.949940e-17,9.981034e-21,5.363301e-17,9.016507e-18
2145,남편/Noun,6.629397e-15,3.642592e-20,5.131438e-18,3.315588e-15
2146,시큼하다/Adjective,2.261535e-15,1.272839e-21,8.036501e-19,1.103470e-17
2147,산모퉁이/Noun,1.493159e-12,2.331639e-18,2.891496e-17,5.686003e-13


In [5]:
whole = pd.read_csv('/content/drive/MyDrive/신입프로젝트/with_score_saved.csv')
whole = whole.drop(columns = ['Unnamed: 0','Unnamed: 0.1','sentiment_eval'])
whole

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,category
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,Unlabeled
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,Unlabeled
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,Unlabeled
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,Unlabeled
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타
...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,Unlabeled
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,Unlabeled
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,Unlabeled


In [6]:
# 테스팅용 function입니다
def keyword_scoring(exp_sent):
  coffeescore = 0
  pricescore = 0
  manscore = 0
  vibescore = 0
  for i, word in enumerate(data['token']):
    if word in exp_sent:
      coffeescore += data['커피/Noun'][i]
      pricescore += data['가격/Noun'][i]
      manscore += data['직원/Noun'][i]
      vibescore += data['분위기/Noun'][i]
  print(coffeescore)
  print(pricescore)
  print(manscore)
  print(vibescore)
  return max([coffeescore, pricescore, manscore, vibescore])

In [7]:
exp_sent = ['선곡/Noun', '좋다/Adjective']

In [8]:
keyword_scoring(exp_sent)    # 대박...선곡 좋다를 분위기로 분류하는 우리의...!!

1.4405356574393e-12
5.690530628740001e-18
1.5848562593999999e-15
3.057836606e-11


3.057836606e-11

In [9]:
# 카테고리 분류에 사용할 함수 정의

def keyword_category(exp_sent):
  coffeescore = 0
  pricescore = 0
  clerkscore = 0
  vibescore = 0
  for i, word in enumerate(data['token']):
    if word in exp_sent:
      coffeescore += data['커피/Noun'][i]
      pricescore += data['가격/Noun'][i]
      clerkscore += data['직원/Noun'][i]
      vibescore += data['분위기/Noun'][i]

  if max([coffeescore, pricescore, clerkscore, vibescore]) == coffeescore:
    return '커피&디저트'
  elif max([coffeescore, pricescore, clerkscore, vibescore]) == pricescore:
    return '가격'
  elif max([coffeescore, pricescore, clerkscore, vibescore]) == clerkscore:
    return '서비스'
  else:
    return '분위기'

In [10]:
# 길이가 하나 이상인 리뷰들에 대해 새로운 칼럼에 카테고리 레이블링

whole['keyword'] = whole['category']
for i in range(len(whole)):
  if whole['LSTM_pp'].isnull()[i] == False:
    if whole['category'].iloc[i] == 'Unlabeled':    
      whole['keyword'].iloc[i] = keyword_category(whole['LSTM_pp'].iloc[i])

whole

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,category,keyword
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,Unlabeled,커피&디저트
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,Unlabeled,커피&디저트
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,Unlabeled,서비스
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,Unlabeled,커피&디저트
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타,기타
...,...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타,기타
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,Unlabeled,커피&디저트
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,Unlabeled,가격
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,Unlabeled,분위기


In [11]:
whole.drop('category',axis = 1, inplace = True)   #임시로 만들었던 'category'는 이제 쓸모없으니 버리기

In [12]:
part = whole[['review','keyword']]
coffee = part[part.keyword == '분위기']    #'분위기'에 해당하는 리뷰들 뽑아보기
for i,sent in enumerate(coffee.review):
  print('{}'.format(sent))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
좋은 분위기 하고 맛있는 카피! 데이드 이나 공부 이나 읽기 잘 하는 곳! 
일단 규모에서 오는 편안함이 있음 일층 이외엔 와이파이 안적혀 있는 듯 도란도란 얘기하기 좋음 규모가 큰데도 희한하게 혼자는 안갈 거 같음
컨테이너카페 좋다
처음갔을 땐 넓고 자리도 많아서 좋았는데 요즘은 항상 만석 ㅠㅠ 
음료랑 분위기는 괜찮은데 2층 티비에 메뉴 나올때마다 울리는 띵동소리가 너무 거슬려요 ㅋㅋㅋㅋ 소리가 너무 커서 공부하기에는 최악입니다 대화나누다가도 흠칫 놀라서 맥끊겨요.. 그리고 화장실도 더러움.. 조리퐁프라페 먹고싶어서 왔는데 다신 안올것같아요 ㅋㅋㅋ
분위기가 좋구요 룸도 깨끗하고 넉넉해요. 음료도 완전 가성비 좋았구.. 무엇보다 대여비가 완전 저렴해요. 다음에 또 이용하여구요~
깔끔하고 조용하세 공부하기 좋네요~
맛있게 달았고, 예뻐서 선물하기 좋았어요.
분위기 인테리어 맛 흠잡을게 없습니다 ㅎㅎ 깔끔하고 맛도 있고 또 가고싶네요
인테리어와 소품 하나하나가 굉장히 신경 쓴 흔적이 보이고 이쁘다. 분위기도 좋고. 3층에다가 통유리에 하얀커튼으로 되어 있어 채광도 좋고 깔끔하고. 가게가 크지 않아 테이블이 많지는 않지만 사람이 적어서 조용하고 좋다. 조명도 굉장히 신경을 쓴거 같은데 따뜻한 느낌이 드는 주광등인거 같고 어두워지면 더 이쁠거 같다. 좀 아쉬운 것은 남여 공동화장실이라는 점...
매장 짱예쁨! 여심저격! 사진찍기딱인곳
그냥 보통. 대학가인만큼 사람 많고 와글와글해요 근데 언제부터 빵빵한 와이파이가 카페의 의무가 됐나요? 충전 코드를 막아놓은 것도 아니고 서비스가 별로도 아니고 와이파이가 부실하다고 1점 주는 수준; 새삼 와이파이 거지들이 많네요?
다른 건 다 모르겠고 00하기만은 진짜 좋은 곳임.. 나만 알아야 하니까 궁금하면 물어봐
자리가 널널하고 간격도 넓고 의자도 넓어서 불편하지 않아요! 좋아유~
따뜻하고 아늑한 느낌에 모임용으로도 정말 좋은것 같아요ㅋㅋㅋ 투썸 좋아하는데 언제 생

In [13]:
whole

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타
...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,커피&디저트
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,가격
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,분위기


In [14]:
whole.keyword.value_counts()     # 카테고리 별 분포 확인

커피&디저트    14411
기타        10195
분위기        6060
서비스        1943
가격         1110
Name: keyword, dtype: int64

In [15]:
whole.keyword.fillna('Unknown',inplace = True)     # 영어, 일본어 등등 한국어 전처리 불가한 리뷰들은 키워드 칼럼에 unknown으로 표기

In [16]:
whole

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타
...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,커피&디저트
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,가격
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,분위기


In [17]:
whole.info()      #이제 keyword 칼럼에 난값 없다!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34489 entries, 0 to 34488
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   row_num    34489 non-null  int64 
 1   location   34489 non-null  object
 2   cafe_name  34489 non-null  object
 3   review     34489 non-null  object
 4   LSTM_pp    33719 non-null  object
 5   label_org  34489 non-null  int64 
 6   label_mod  34489 non-null  int64 
 7   LDA_pp     33645 non-null  object
 8   keyword    34489 non-null  object
dtypes: int64(3), object(6)
memory usage: 2.4+ MB


In [18]:
whole.keyword.value_counts()     #마지막으로 확인

커피&디저트     14411
기타         10195
분위기         6060
서비스         1943
가격          1110
Unknown      770
Name: keyword, dtype: int64

## CSV 저장 코드

In [19]:
whole.to_csv('/content/drive/MyDrive/신입프로젝트/keyword_labeled_whole_data.csv')